In [16]:
from darknet import parse_cfg, create_modules, DarkNet, get_test_input
from util import load_classes, write_results, letterbox_image, prep_image, write, display_img, rescale
import torch.nn as nn
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random

In [17]:
#Load model configuration
blocks = parse_cfg('C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3/cfg/yolov3.cfg')
net_info, module_list = create_modules(blocks)

In [19]:
#Load classes
classes = load_classes("coco.names")
#Load colours
colours = pickle.load(open("pallete","rb"))

In [23]:
#Get path of video
root = "C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3 - Video/Core/videos/"
path = os.path.join(root,'highway traffic.mp4')
video = cv2.VideoCapture(path)
if video.isOpened():
    pass
else:
    print("Video cannot be opened")

In [24]:
#Intialize model
model = DarkNet('C:/Users/Joab-PC/OneDrive - Nanyang Technological University/NTU/2020 (SEM 2)/FYP/YOLOv3/cfg/yolov3.cfg')
#Load weights
model.load_weights("yolov3.weights")
model = model.cuda()

In [25]:
counter = 0
while video.isOpened():
    ret, frame = video.read()
    if ret:
        #Prepare image
        resized_image = letterbox_image(frame, (416,416))
        processed_image = prep_image(resized_image.astype('float32'),416)
        im_dim = frame.shape[1] , frame.shape[0]
        
        #Detection
        model.eval()
        pred = model(processed_image.cuda())
        outputs = write_results(pred,confidence=0.3, num_classes=80)
        
        #Rescaling
        rescaled_outputs = rescale(outputs,im_dim)
        
        img_bounding_box = cv2.rectangle(frame,(134,536), (483,702),1)

        for output in rescaled_outputs:
            #Draw bounding boxes
            c1 = tuple(output[1:3].int().cpu().numpy())
            c2 = tuple(output[3:5].int().cpu().numpy())
            colour = random.choice(colours)
            img_bounding_box = cv2.rectangle(frame,c1,c2,colour)
            #Get class and accuracy
            cls = int(output[-1])
            acc = output[-2]

            #Put label onto image
            label = "{} {:2}".format(classes[cls], round(acc.item(),2))
            font                   = cv2.FONT_HERSHEY_SIMPLEX
            fontScale              = 0.5
            fontColor              = 1
            lineType               = 2

            img_bounding_box = cv2.putText(frame,label,c1,font,fontScale,fontColor,lineType)
            
        cv2.imshow("frame", frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

KeyboardInterrupt: 

In [ ]:
right_clicks = []
def mouse_callback(event, x, y, flags, params):

    #right-click event value is 2
    if event == 2:
        global right_clicks

        #store the coordinates of the right-click event
        right_clicks.append([x, y])

        #this just verifies that the mouse data is being collected
        #you probably want to remove this later
        print(right_clicks)

In [ ]:
ret, frame = video.read()
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('image', mouse_callback)
cv2.imshow('image', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()